In [63]:
def get_db_schema(db_name):
    """Retrieve schema information for all objects (tables, indexes, views, triggers) in the SQLite database."""
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Define a function to get schema information for tables
    def get_table_schema(table_name):
        cursor.execute(f"PRAGMA table_info({table_name});")
        schema = cursor.fetchall()
        return pd.DataFrame(schema, columns=['Column ID', 'Column Name', 'Data Type', 'Not Null', 'Default Value', 'Primary Key'])
    
    # Define a function to get schema information for indexes
    def get_index_schema(index_name):
        cursor.execute(f"PRAGMA index_info({index_name});")
        schema = cursor.fetchall()
        return pd.DataFrame(schema, columns=['Column ID', 'Column Name', 'Sort Order'])
    
    # Define a function to get schema information for views
    def get_view_schema(view_name):
        cursor.execute(f"SELECT sql FROM sqlite_master WHERE type='view' AND name='{view_name}';")
        schema = cursor.fetchone()
        return pd.DataFrame([schema], columns=['View SQL'])
    
    # Define a function to get schema information for triggers
    def get_trigger_schema(trigger_name):
        cursor.execute(f"SELECT sql FROM sqlite_master WHERE type='trigger' AND name='{trigger_name}';")
        schema = cursor.fetchone()
        return pd.DataFrame([schema], columns=['Trigger SQL'])
    
    # Initialize the dictionary to hold schema information
    schema_dict = {'tables': {}, 'indexes': {}, 'views': {}, 'triggers': {}}
    
    # Get and store schema information for tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    for table in [t[0] for t in tables]:
        schema_dict['tables'][table] = get_table_schema(table)
    
    # Get and store schema information for indexes
    cursor.execute("SELECT name FROM sqlite_master WHERE type='index';")
    indexes = cursor.fetchall()
    for index in [i[0] for i in indexes]:
        schema_dict['indexes'][index] = get_index_schema(index)
    
    # Get and store schema information for views
    cursor.execute("SELECT name FROM sqlite_master WHERE type='view';")
    views = cursor.fetchall()
    for view in [v[0] for v in views]:
        schema_dict['views'][view] = get_view_schema(view)
    
    # Get and store schema information for triggers
    cursor.execute("SELECT name FROM sqlite_master WHERE type='trigger';")
    triggers = cursor.fetchall()
    for trigger in [t[0] for t in triggers]:
        schema_dict['triggers'][trigger] = get_trigger_schema(trigger)
    
    conn.close()
    
    # Return the complete dictionary with all object types
    return schema_dict


In [64]:
b3 = get_db_schema('b3.db')

In [74]:
def load_database(db_name):
    """Load each table from the SQLite database into its own DataFrame."""
    conn = sqlite3.connect(db_name)
    
    # Define a function to get table names
    def get_table_names():
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        return [table[0] for table in tables]
    
    # Initialize a dictionary to hold DataFrames for each table
    table_dfs = {}
    
    # Get the list of table names
    table_names = get_table_names()
    
    # Load each table into its own DataFrame
    for table_name in table_names:
        query = f"SELECT * FROM {table_name};"
        table_dfs[table_name] = pd.read_sql_query(query, conn)
    
    conn.close()
    
    return table_dfs



In [77]:
b3 = load_database('b3.db')
nsd = b3['nsd']
nsd

,nsd,company,dri,nsd_type,version,auditor,auditor_rt,protocolo,date,sent_date,reason
0,34,BAESAENERGETICA BARRA GRANDE SA,CARLOS ALBERTO BEZERRA DE MIRANDA,FORMULARIO CADASTRAL,1,KPMG AUDITORES INDEPENDENTES,MARCELO LIMA TONINI,019607FCA00002010010000003487,2010-01-01T00:00:00,2010-03-05T16:22:04,
1,35,PRUMO LOGISTICA SA,OTAVIO DE GARCIA LAZCANO,FORMULARIO CADASTRAL,1,KPMG AUDITORES INDEPENDENTES,MANUEL FERNANDES RODRIGUES DE SOUSA,021482FCA00002010010000003588,2010-01-01T00:00:00,2010-03-05T16:35:55,
2,36,CIA SANEAMENTO BASICO EST SAO PAULO,RUI DE BRITTO ALVARES AFFONSO,FORMULARIO CADASTRAL,1,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES,PAULO CESAR ESTEVAO NETTO,014443FCA00002010010000003680,2010-01-01T00:00:00,2010-03-05T18:41:51,
3,37,EVEN CONSTRUTORA E INCORPORADORA SA,DANY MUSZKAT,FORMULARIO CADASTRAL,1,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES,VALDIR RENATO COSCODAI,020524FCA00002010010000003782,2010-01-01T00:00:00,2010-03-09T17:00:41,
4,39,JBS SA,JEREMIAH ALPHONSUS OCALLAGHAN,FORMULARIO CADASTRAL,1,BDO TREVISAN AUDITORES INDEPENDENTES,ESTEFAN GEORGE HADDAD,020575FCA00002010010000003982,2010-01-01T00:00:00,2010-03-11T20:02:44,
...,...,...,...,...,...,...,...,...,...,...,...
28178,30270,FRASLE SA,DANIEL RAUL RANDON,INFORMACOES TRIMESTRAIS,1,ERNST YOUNG AUDITORES INDEPENDENTES SS,AMERICO FFERREIRA NETO,006211ITR30062013010003027070,2013-06-30T00:00:00,2013-08-07T17:32:05,
28179,30271,CONSTRUTORA ADOLPHO LINDENBERG SA,ADOLPHO LINDENBERG FILHO,INFORMACOES TRIMESTRAIS,1,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENTES,WALTER DALSASSO,004723ITR30062013010003027176,2013-06-30T00:00:00,2013-08-07T17:33:54,
28180,30272,LINX SA,DENNIS HERSZKOWICZ,INFORMACOES TRIMESTRAIS,1,KPMG,WAGNER PETELIN,023035ITR30062013010003027279,2013-06-30T00:00:00,2013-08-07T17:38:47,
28181,30273,REC SAO BERNARDO SECURITIZADORA DE CREDITOS IM...,PEDRO MARCIO DALTRO DOS SANTOS,FORMULARIO DE REFERENCIA,3,ERNST YOUNG AUDITORES INDEPENDENTES SS,ACYR DE OLIVEIRA PEREIRA,019356FRE20132013030003027362,2013-01-01T00:00:00,2013-08-07T17:52:40,RENUNCIA DE DIRETORES


In [70]:
# Displaying the first few rows of each DataFrame
for table_name, df in table_dfs.items():
    print(f"Table: {table_name}")
    display(df.head())  # Using display() in Jupyter Notebook to show DataFrames


Table: nsd


,nsd,company,dri,nsd_type,version,auditor,auditor_rt,protocolo,date,sent_date,reason
0,34,BAESAENERGETICA BARRA GRANDE SA,CARLOS ALBERTO BEZERRA DE MIRANDA,FORMULARIO CADASTRAL,1,KPMG AUDITORES INDEPENDENTES,MARCELO LIMA TONINI,019607FCA00002010010000003487,2010-01-01T00:00:00,2010-03-05T16:22:04,
1,35,PRUMO LOGISTICA SA,OTAVIO DE GARCIA LAZCANO,FORMULARIO CADASTRAL,1,KPMG AUDITORES INDEPENDENTES,MANUEL FERNANDES RODRIGUES DE SOUSA,021482FCA00002010010000003588,2010-01-01T00:00:00,2010-03-05T16:35:55,
2,36,CIA SANEAMENTO BASICO EST SAO PAULO,RUI DE BRITTO ALVARES AFFONSO,FORMULARIO CADASTRAL,1,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES,PAULO CESAR ESTEVAO NETTO,014443FCA00002010010000003680,2010-01-01T00:00:00,2010-03-05T18:41:51,
3,37,EVEN CONSTRUTORA E INCORPORADORA SA,DANY MUSZKAT,FORMULARIO CADASTRAL,1,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES,VALDIR RENATO COSCODAI,020524FCA00002010010000003782,2010-01-01T00:00:00,2010-03-09T17:00:41,
4,39,JBS SA,JEREMIAH ALPHONSUS OCALLAGHAN,FORMULARIO CADASTRAL,1,BDO TREVISAN AUDITORES INDEPENDENTES,ESTEFAN GEORGE HADDAD,020575FCA00002010010000003982,2010-01-01T00:00:00,2010-03-11T20:02:44,


In [59]:
import sqlite3
import pandas as pd

def load_database(db_name='b3.db'):
    """
    Carrega os dados do banco de dados b3.db em um DataFrame do Pandas.

    Parameters:
    - db_name (str): O nome do banco de dados SQLite.

    Returns:
    pd.DataFrame: DataFrame contendo os dados da tabela nsd.
    """
    # Conecte-se ao banco de dados
    conn = sqlite3.connect(db_name)
    
    # Leia os dados da tabela nsd
    query = "SELECT * FROM nsd"
    df = pd.read_sql_query(query, conn)
    
    # Feche a conexão
    conn.close()
    
    return df



In [61]:
# Use a função para carregar os dados
df_nsd = load_database()
df_nsd

,nsd,company,dri,nsd_type,version,auditor,auditor_rt,protocolo,date,sent_date,reason
0,34,BAESAENERGETICA BARRA GRANDE SA,CARLOS ALBERTO BEZERRA DE MIRANDA,FORMULARIO CADASTRAL,1,KPMG AUDITORES INDEPENDENTES,MARCELO LIMA TONINI,019607FCA00002010010000003487,2010-01-01T00:00:00,2010-03-05T16:22:04,
1,35,PRUMO LOGISTICA SA,OTAVIO DE GARCIA LAZCANO,FORMULARIO CADASTRAL,1,KPMG AUDITORES INDEPENDENTES,MANUEL FERNANDES RODRIGUES DE SOUSA,021482FCA00002010010000003588,2010-01-01T00:00:00,2010-03-05T16:35:55,
2,36,CIA SANEAMENTO BASICO EST SAO PAULO,RUI DE BRITTO ALVARES AFFONSO,FORMULARIO CADASTRAL,1,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES,PAULO CESAR ESTEVAO NETTO,014443FCA00002010010000003680,2010-01-01T00:00:00,2010-03-05T18:41:51,
3,37,EVEN CONSTRUTORA E INCORPORADORA SA,DANY MUSZKAT,FORMULARIO CADASTRAL,1,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES,VALDIR RENATO COSCODAI,020524FCA00002010010000003782,2010-01-01T00:00:00,2010-03-09T17:00:41,
4,39,JBS SA,JEREMIAH ALPHONSUS OCALLAGHAN,FORMULARIO CADASTRAL,1,BDO TREVISAN AUDITORES INDEPENDENTES,ESTEFAN GEORGE HADDAD,020575FCA00002010010000003982,2010-01-01T00:00:00,2010-03-11T20:02:44,
...,...,...,...,...,...,...,...,...,...,...,...
28178,30270,FRASLE SA,DANIEL RAUL RANDON,INFORMACOES TRIMESTRAIS,1,ERNST YOUNG AUDITORES INDEPENDENTES SS,AMERICO FFERREIRA NETO,006211ITR30062013010003027070,2013-06-30T00:00:00,2013-08-07T17:32:05,
28179,30271,CONSTRUTORA ADOLPHO LINDENBERG SA,ADOLPHO LINDENBERG FILHO,INFORMACOES TRIMESTRAIS,1,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENTES,WALTER DALSASSO,004723ITR30062013010003027176,2013-06-30T00:00:00,2013-08-07T17:33:54,
28180,30272,LINX SA,DENNIS HERSZKOWICZ,INFORMACOES TRIMESTRAIS,1,KPMG,WAGNER PETELIN,023035ITR30062013010003027279,2013-06-30T00:00:00,2013-08-07T17:38:47,
28181,30273,REC SAO BERNARDO SECURITIZADORA DE CREDITOS IM...,PEDRO MARCIO DALTRO DOS SANTOS,FORMULARIO DE REFERENCIA,3,ERNST YOUNG AUDITORES INDEPENDENTES SS,ACYR DE OLIVEIRA PEREIRA,019356FRE20132013030003027362,2013-01-01T00:00:00,2013-08-07T17:52:40,RENUNCIA DE DIRETORES
